In [1986]:
# City School Analysis - Written by Aruna Amaresan 
# Date: Dec 07 - Dec 09th 2017
# For slicing and dicing All Schools in Districts, Do School Performance Analysis Data 
# Assumptions: For Math:    The pass mark is calculated for 60 and above 
#              For Reading: The pass mark is calculated for 65 and above 
# Input: The sources CSV files are expected to be kept in the raw_data folder - one for student and other for school
# 
# Output: 9 different reports are generated: - District Summary 
#                                            - School Summary ( Aggregated within each School)
#                                            - Top 5 Performing Schools by % Overall Passing Rate ( Aggregated within each School)     
#                                            - Bottom 5 Performing Schools by % Overall Passing Rate ( Aggregated within each School)
#                                            - Math Score (Averages) by Grade
#                                            - Reading Score (Averages) by Grade - 9th to 12th 
#                                            - Performances by School Spending 
#                                            - Performances by School Size
#                                            - Performances by School Type

# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os 

In [1987]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

#print (color.BOLD + 'Hello World !' + color.END)

In [1988]:
# load CSV files 
schoolsCSV = os.path.join('raw_data', 'schools_complete.csv')

schools_pd = pd.read_csv(schoolsCSV, encoding="iso-8859-1", low_memory=False)
#schools_pd.head(20)

In [1989]:
#Load Student CSV file 
studentsCSV = os.path.join('raw_data', 'students_complete.csv')

students_pd = pd.read_csv(studentsCSV, encoding="iso-8859-1", low_memory=False)
#students_pd.head()

In [1990]:
#cleanse of any empty rows in students
#students_pd.columns
students = students_pd.dropna(how="any")
#students.head()

In [1991]:
#Clean up any rows with empty values in schools data
school_count = schools_pd.dropna(how="any")

#school_count.head()

In [1992]:
#school_count.columns
#students.columns

In [1993]:
#Get Total Number of school 
NumberOfSchools = len(school_count)
#print(NumberOfSchools)


#get Total number of students 
TotalStudents = school_count['size'].sum()
#print(TotalStudents)

#Get Total Budget
TotalBudget = school_count['budget'].sum()
#print(TotalBudget)

In [1994]:
#student_schoolcount = students_pd["school"].value_counts()
#student_schoolcount

#Find average math score 
average_MathScore = students["math_score"].mean()
#print(average_MathScore)

#Find average reading score 
average_ReadingScore = students["reading_score"].mean()
#print(average_ReadingScore)

#Lets assume a passing score is 60 and above for Math 
passed_math = students.loc[students["math_score"] >= 60]
#passed_math.head()

#Find Percent of Students who had passed math
passed_math_count = passed_math["name"].count()
#print(passed_math_count)

#For reading calculate the same - Lets assume a passing rate of 65 for reading
passed_reading = students.loc[students["reading_score"] >= 65]
#print(passed_reading.head())

#Find Percent of Students who had passed math
passed_reading_count = passed_reading["name"].count()
#print(passed_reading_count)

In [1995]:
# Calculate percentage of students passign math and similarly for reading . Then calculate overall percentage 
math_percent = (passed_math_count/TotalStudents) * 100
#print(math_percent)
reading_percent = (passed_reading_count/TotalStudents) * 100
#print(reading_percent)

overall_passing_rate = ((math_percent + reading_percent)/2) 
#print( overall_passing_rate )
#print(f" % Male: {male_percent}\n % Female: {female_percent}\n % non_specifc: {non_gender_specific}")

In [1996]:
# Create a new table consolodating above calculations
district_summary = pd.DataFrame({"Total Schools": [NumberOfSchools],
                                   "Total Students": [TotalStudents],
                                   "Total Budget": [TotalBudget],
                                   "Average Math Score": [average_MathScore],
                                   "Average Reading Score": [average_ReadingScore],
                                   "% Passing Math":[math_percent],
                                   "% Passing Reading":[reading_percent],
                                   "% Overall Passing Rate": [ overall_passing_rate ]
})
district_summary  = district_summary [["Total Schools",
                                   "Total Students",
                                   "Total Budget",
                                   "Average Math Score",
                                   "Average Reading Score",
                                   "% Passing Math",
                                   "% Passing Reading",
                                   "% Overall Passing Rate"]]

district_summary  = district_summary.round(2)

#district_summary 

In [1997]:
# Formatting to be done to certain columns as per sample PDF shown 
district_summary["Total Students"] = district_summary["Total Students"].map("{0:,.0f}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${0:,.2f}".format)
print (color.BOLD + 'District Summary' + color.END)
print ('****************')
district_summary
#district_summary.dtypes

District Summary
****************


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,92.45,96.2,94.32


In [1998]:
#For School Summary 

# Take only columns we need from school_count
reduced_school_count = school_count.loc[:, ['name','type','size', 'budget']]
#reduced_school_count.head()

In [1999]:
reduced_school_count = reduced_school_count.rename(columns={"name":"school", 
                                                            "type":"School Type", 
                                                            "size":"Total Students", 
                                                            "budget":"Total School Budget",
                                                                 })
#reduced_school_count

#Store data for binning on School Size and School Type
School_Size_Type_Performance = pd.DataFrame(reduced_school_count)


In [2000]:
# Calculate percentage of eac bootcamp's students who are recommenders
reduced_school_count["Per Student Budget"] = reduced_school_count["Total School Budget"] / reduced_school_count["Total Students"]

#reduced_school_count.head()

In [2001]:
# Take only columns we need from school_count
#name	gender	grade	school	reading_score	math_score
reduced_student_columns = students.loc[:, ['school','reading_score', 'math_score']]
#reduced_student_columns.head()



In [2002]:
# Group By Schools and Find Average reading score
AverageMathScore_BySchool = pd.DataFrame(reduced_student_columns.groupby("school")["math_score"].mean())

AverageMathScore_BySchool.reset_index(inplace=True)
AverageMathScore_BySchool.columns=["school", "Average Math Score"]

AverageMathScore_BySchool.head()

In [2003]:
# Create a groupby variable that groups preTestScores by regiment
#groupby_schools = students['reading_score'].groupby(students['school'])
#groupby_schools.mean()

# Group By Schools and Find Average reading score
AverageReadingScore_BySchool = pd.DataFrame(reduced_student_columns.groupby("school")["reading_score"].mean())

AverageReadingScore_BySchool.reset_index(inplace=True)
AverageReadingScore_BySchool.columns=["school", "Average Reading Score"]

#AverageReadingScore_BySchool.head()

In [2004]:
# Merge the two created data frames on the name of tbe school
merged_scores = pd.merge(AverageMathScore_BySchool , AverageReadingScore_BySchool, on="school")
#merged_scores.head()

#Students_InEachSchool = reduced_student_columns["school"].value_counts()
#Students_InEachSchool

In [2005]:
# The pass score for Math is always assumed to be 60 and
# the pass score for reading is always taken at 65

# Group By Schools and Find Pass Math Count 
BySchool_MathPass = reduced_student_columns[reduced_student_columns['math_score'] >= 60].groupby('school')['math_score'].count()

#Group_BySchool = reduced_student_columns.groupby(["school"])  #grouped_usa_df = usa_ufo_df.groupby(["state"])
BySchool_MathPassCount = pd.DataFrame (BySchool_MathPass)
BySchool_MathPassCount.reset_index(level = 0, inplace = True) # df.reset_index(level=0, inplace=True)
#BySchool_MathPassCount.columns = ["school", "Math Pass Count "]
#BySchool_MathPassCount.head()


In [2006]:
#Now find the Reading Pass Count 
BySchool_ReadPass = reduced_student_columns[reduced_student_columns['reading_score'] >= 65].groupby('school')['reading_score'].count()

#Group_BySchool = reduced_student_columns.groupby(["school"])  
BySchool_ReadPassCount = pd.DataFrame (BySchool_ReadPass)
BySchool_ReadPassCount.reset_index(level = 0, inplace = True) # Ensure we make the index also a column 

#BySchool_ReadPassCount.head()


In [2007]:
#Merge the Pass Data Frames on the name of school
merged_PassCount = pd.merge(BySchool_MathPassCount , BySchool_ReadPassCount, on="school")
#merged_PassCount.head()

In [2008]:
#Merge the Score and Pass Data Frames on the name of school
merged_Scores_PassCount = pd.merge(merged_scores , merged_PassCount, on="school")
#merged_Scores_PassCount.head()

In [2009]:
merged_school_studentData = pd.merge(reduced_school_count , merged_Scores_PassCount, on="school")
#merged_school_studentData.head()

In [2010]:
# Calculate percentage of Math Pass Rate and add to column 
merged_school_studentData["% Passing Math"] = merged_school_studentData["math_score"] / merged_school_studentData["Total Students"] * 100

# Sort results in descending order
#merged_df = merged_df.sort_values(["% Recommend"], ascending=False).round(2)

#merged_school_studentData.head()

In [2011]:
# Calculate percentage of Math Pass Rate and add to column 
merged_school_studentData["% Passing Reading"] = merged_school_studentData["reading_score"] / merged_school_studentData["Total Students"] * 100

# Sort results in descending order
#merged_df = merged_df.sort_values(["% Recommend"], ascending=False).round(2)


#merged_school_studentData.head()

In [2012]:
# Calculate percentage of Math Pass Rate and add to column 
merged_school_studentData["% Overall Passing Rate"] = (merged_school_studentData["% Passing Math"] + merged_school_studentData["% Passing Reading"]) / 2
#merged_school_studentData.dtypes

#Store data away for Binning Exercises 
PerStudentSpending_BySchool = pd.DataFrame(merged_school_studentData)

merged_school_studentData.drop(['math_score'], axis = 1, inplace = True)
merged_school_studentData.drop(['reading_score'], axis = 1, inplace = True)


#For Display of the values - store in a sperate Data Frame
display_school_studentData = pd.DataFrame(merged_school_studentData)

# Format for $ values for the Budget columns
display_school_studentData["Per Student Budget"] = display_school_studentData["Per Student Budget"].map("${0:,.2f}".format)
display_school_studentData["Total School Budget"] = display_school_studentData["Total School Budget"].map("${0:,.2f}".format)


# Format for percentages

display_school_studentData = display_school_studentData.rename(columns={"school":""})
#display_school_studentData.head()

display_school_studentData["% Passing Reading"] = display_school_studentData["% Passing Reading"].map("{0:,.2f}%".format)
# Format for percentages
display_school_studentData["% Passing Math"] = display_school_studentData["% Passing Math"].map("{0:,.2f}%".format)

display_school_studentData["% Overall Passing Rate"] = display_school_studentData["% Overall Passing Rate"].map("{0:,.2f}%".format)


print (color.BOLD + "School Summary" + color.END)
print ("***************")
display_school_studentData


#merged_school_studentData.head()



#Drop the unneded columns and pill in only what u need for School Summary 
#final_students_data = merged_school_studentData.iloc[:,[0,1,2,3,4,5,6,9,10,11]]

#final_students_data.reset_index(level=0, inplace=True)
#final_students_data.head()

School Summary
***************


,,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,88.86%,94.48%,91.67%
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,88.44%,94.54%,91.49%
2,Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,100.00%,100.00%,100.00%
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,89.08%,94.61%,91.84%
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,100.00%,100.00%,100.00%
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,100.00%,100.00%,100.00%
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,100.00%,100.00%,100.00%
7,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,89.53%,94.55%,92.04%
8,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,100.00%,100.00%,100.00%
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,100.00%,100.00%,100.00%


In [2013]:
#Display top 5 entries 
#merged_school_studentData["% Overall Passing Rate"] = pd.to_numeric(merged_school_studentData["% Overall Passing Rate"])

#Get Data for Top 5 and botttom 5 performing schools - Sort results in descending order
#sorted_by_PassRate = 

#sorted_by_PassRate.head()

sorted_by_PassRate = merged_school_studentData.sort_values("% Overall Passing Rate", ascending = False).round(2)

sorted_by_PassRate.set_index('school', inplace = True)

sorted_by_PassRate = sorted_by_PassRate.rename(index={'school':''})
sorted_by_PassRate

top_performing_header = 'Top Performing Schools (By Passing Rate)\n'
print (color.BOLD + top_performing_header + color.END)
print('*****************************************')

display_Top5PassRate = sorted_by_PassRate.iloc[0:5,:]

# Format for $ values for the Budget columns
#display_Top5PassRate["Per Student Budget"] = display_Top5PassRate["Per Student Budget"].map("${0:,.2f}".format)
#display_Top5PassRate["Total School Budget"] = display_Top5PassRate["Total School Budget"].map("${0:,.2f}".format)


display_Top5PassRate
#sorted_by_PassRate = merged_school_studentData.sort_values("% Overall Passing Rate"", ascending=False)
#sorted_by_PassRate.head()

Top Performing Schools (By Passing Rate)

*****************************************


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school,,,,,,,,,
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.36,83.73,100.0,100.0,100.0
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,100.0,100.0,100.0
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,100.0,100.0,100.0
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,100.0,100.0,100.0
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,100.0,100.0,100.0


In [2014]:
#sorted_by_PassRate.columns

sorted_by_PassRate = merged_school_studentData.sort_values("% Overall Passing Rate", ascending = True).round(2)

sorted_by_PassRate.set_index('school', inplace = True)

sorted_by_PassRate = sorted_by_PassRate.rename(index={'school':''})
#sorted_by_PassRate

bottom_performing_header = 'Bottom Performing Schools (By Passing Rate)\n'
print (color.BOLD + bottom_performing_header + color.END)
print('********************************************')

#get Last 5 rows of dataframe to Display Bottom performing Schools by passing rate 
display_Bottom5PassRate = sorted_by_PassRate.iloc[0:5,:] 
display_Bottom5PassRate

Bottom Performing Schools (By Passing Rate)

********************************************


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school,,,,,,,,,
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,88.44,94.54,91.49
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,89.30,93.87,91.58
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,88.55,94.62,91.59
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,88.86,94.48,91.67
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,89.18,94.48,91.83


In [2015]:
# For Math Scrores (Average) By Grade 
#name	gender	grade	school	reading_score	math_score 
reduced_student_mathcolumns = students.loc[:, ['school','grade', 'math_score']]
#reduced_student_mathcolumns.columns
#reduced_student_mathcolumns.dtypes

#MathScore_ByGradeSchool = pd.DataFrame(reduced_student_mathcolumns.groupby(["school", "grade"])["math_score"].mean())
#MathScore_ByGradeSchool

#unique = reduced_student_mathcolumns["grade"].unique()
#unique

In [2016]:
#Get 9th grade Average Math score by School 
Stu_9thGrade_Math = reduced_student_mathcolumns[reduced_student_mathcolumns['grade'] == '9th'].groupby('school')['math_score'].mean()
Stu_9thGrade_MathAvg = pd.DataFrame (Stu_9thGrade_Math)
Stu_9thGrade_MathAvg.reset_index(level = 0, inplace = True) # df.reset_index(level=0, inplace=True)
Stu_9thGrade_MathAvg = Stu_9thGrade_MathAvg.rename(columns={"math_score":"9th"})
#Stu_9thGrade_MathAvg.head()


In [2017]:
#Get 10th grade Average Math score by School 
Stu_10thGrade_Math = reduced_student_mathcolumns[reduced_student_mathcolumns['grade'] == '10th'].groupby('school')['math_score'].mean()
Stu_10thGrade_MathAvg = pd.DataFrame (Stu_10thGrade_Math)

Stu_10thGrade_MathAvg.reset_index(level = 0, inplace = True) # df.reset_index(level=0, inplace=True)
Stu_10thGrade_MathAvg = Stu_10thGrade_MathAvg.rename(columns={"math_score":"10th"})
#Stu_10thGrade_MathAvg.head()



In [2018]:
#Get 11th grade Average Math score by School 
Stu_11thGrade_Math = reduced_student_mathcolumns[reduced_student_mathcolumns['grade'] == '11th'].groupby('school')['math_score'].mean()
Stu_11thGrade_MathAvg = pd.DataFrame (Stu_11thGrade_Math)

Stu_11thGrade_MathAvg.reset_index(level = 0, inplace = True) # df.reset_index(level=0, inplace=True)
Stu_11thGrade_MathAvg = Stu_11thGrade_MathAvg.rename(columns={"math_score":"11th"})
#Stu_11thGrade_MathAvg.head()

In [2019]:
#Get 12th grade Average Math score by School 
Stu_12thGrade_Math = reduced_student_mathcolumns[reduced_student_mathcolumns['grade'] == '12th'].groupby('school')['math_score'].mean()
Stu_12thGrade_MathAvg = pd.DataFrame (Stu_12thGrade_Math)

Stu_12thGrade_MathAvg.reset_index(level = 0, inplace = True) # df.reset_index(level=0, inplace=True)
Stu_12thGrade_MathAvg = Stu_12thGrade_MathAvg.rename(columns={"math_score":"12th"})
#Stu_12thGrade_MathAvg.head()

In [2020]:
#Merge all the 9th 10h 11th and 12th average Math grade by school into one table 
merged_9th_10th_AvgMathScore = pd.merge(Stu_9thGrade_MathAvg , Stu_10thGrade_MathAvg, on="school")

merged_9th_10th_11thAvgMathScore = pd.merge(merged_9th_10th_AvgMathScore , Stu_11thGrade_MathAvg, on="school")

merged_9th_10th_11th_12thAvgMathScore = pd.merge(merged_9th_10th_11thAvgMathScore , Stu_12thGrade_MathAvg, on="school")

#merged_9th_10th_11th_12thAvgMathScore.head()

In [2021]:
#Update displays to show in form of table 
display_AvgMathScore_BySchoolGrade = merged_9th_10th_11th_12thAvgMathScore
MathGrade_header ="Math Scores By Grade"

display_AvgMathScore_BySchoolGrade = display_AvgMathScore_BySchoolGrade.rename(columns={'school':''})

print (color.BOLD + MathGrade_header + color.END)
print('**********************')
display_AvgMathScore_BySchoolGrade

Math Scores By Grade
**********************


,,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


In [2022]:
# For Reading Scrores (Average) By Grade - Extract the needed columns alone 
#name	gender	grade	school	reading_score	math_score 
reduced_student_readingcolumns = students.loc[:, ['school','grade', 'reading_score']]
#reduced_student_readingcolumns.columns
#reduced_student_readingcolumns.dtypes

#reduced_student_readingcolumns

In [2023]:
#Get 9th grade Average Reading score by School 
Stu_9thGrade_Reading = reduced_student_readingcolumns[reduced_student_readingcolumns['grade'] == '9th'].groupby('school')['reading_score'].mean()
Stu_9thGrade_ReadingAvg = pd.DataFrame (Stu_9thGrade_Reading)
Stu_9thGrade_ReadingAvg.reset_index(level = 0, inplace = True) # df.reset_index(level=0, inplace=True)
Stu_9thGrade_ReadingAvg = Stu_9thGrade_ReadingAvg.rename(columns={"reading_score":"9th"})
#Stu_9thGrade_ReadingAvg.head()

In [2024]:
#Get 10th grade Average Reading score by School 
Stu_10thGrade_Reading = reduced_student_readingcolumns[reduced_student_readingcolumns['grade'] == '10th'].groupby('school')['reading_score'].mean()
Stu_10thGrade_ReadingAvg = pd.DataFrame (Stu_10thGrade_Reading)

Stu_10thGrade_ReadingAvg.reset_index(level = 0, inplace = True) # df.reset_index(level=0, inplace=True)
Stu_10thGrade_ReadingAvg = Stu_10thGrade_ReadingAvg.rename(columns={"reading_score":"10th"})
#Stu_10thGrade_ReadingAvg.head()

In [2025]:
#Get 11th grade Average Reading score by School 
Stu_11thGrade_Reading = reduced_student_readingcolumns[reduced_student_readingcolumns['grade'] == '11th'].groupby('school')['reading_score'].mean()
Stu_11thGrade_ReadingAvg = pd.DataFrame (Stu_11thGrade_Reading)

Stu_11thGrade_ReadingAvg.reset_index(level = 0, inplace = True) # df.reset_index(level=0, inplace=True)
Stu_11thGrade_ReadingAvg = Stu_11thGrade_ReadingAvg.rename(columns={"reading_score":"11th"})
#Stu_11thGrade_ReadingAvg.head()

In [2026]:
#Get 12th grade Average Reading score by School 
Stu_12thGrade_Reading = reduced_student_readingcolumns[reduced_student_readingcolumns['grade'] == '12th'].groupby('school')['reading_score'].mean()
Stu_12thGrade_ReadingAvg = pd.DataFrame (Stu_12thGrade_Reading)

Stu_12thGrade_ReadingAvg.reset_index(level = 0, inplace = True) # df.reset_index(level=0, inplace=True)
Stu_12thGrade_ReadingAvg = Stu_12thGrade_ReadingAvg.rename(columns={"reading_score":"12th"})
#Stu_12thGrade_ReadingAvg.head()

In [2027]:
#Merge all the 9th 10h 11th and 12th average Reading grade by school into one table 
merged_9th_10th_AvgReadScore = pd.merge(Stu_9thGrade_ReadingAvg , Stu_10thGrade_ReadingAvg, on="school")

merged_9th_10th_11thAvgReadScore = pd.merge(merged_9th_10th_AvgReadScore , Stu_11thGrade_ReadingAvg, on="school")

merged_9th_10th_11th_12thAvgReadScore = pd.merge(merged_9th_10th_11thAvgReadScore , Stu_12thGrade_ReadingAvg, on="school")

#merged_9th_10th_11th_12thAvgReadScore.head()

In [2028]:
#Update displays to show in form of table 
display_AvgReadScore_BySchoolGrade = merged_9th_10th_11th_12thAvgReadScore
MathGrade_header ="Reading Scores By Grade"

display_AvgReadScore_BySchoolGrade = display_AvgReadScore_BySchoolGrade.rename(columns={'school':''})

print (color.BOLD + MathGrade_header + color.END)
print('**********************')
display_AvgReadScore_BySchoolGrade

Reading Scores By Grade
**********************


,,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


In [2029]:
# Use the mereged school_student Data Table - First, Extract the columns you need  

PerStudentSpending_BySchool.head()

reduced_PerStudentSpending = PerStudentSpending_BySchool.loc[:, ['school','Per Student Budget', 
                                                               'Average Math Score', 'Average Reading Score',
                                                               '% Passing Math', '% Passing Reading', 
                                                               '% Overall Passing Rate']]
#reduced_PerStudentSpending.head()


In [2030]:
#Parse it to find max per Student Budget and min per Student Budget 
#print(reduced_PerStudentSpending["Per Student Budget"].max())

#print(reduced_PerStudentSpending["Per Student Budget"].min())

In [2031]:
#Put then in Bins and find average for each data within those bins. TO do this: 

#First - Create bins in which to place values based upon Per Student Spending 
bins = [0,585,615,645,675]

# Second - Create labels for these bins
group_labels = ["< $585","$585-615","$615-645","$645-675"]

In [2032]:
# Third - Slice the Per Student Budget Data and place it into bins
#pd.cut(reduced_PerStudentSpending["Per Student Budget"], bins, labels=group_labels).head()

In [2033]:
# Fourth - Place the data series into a new column inside of the DataFrame
reduced_PerStudentSpending["Spending Ranges (Per Student)"] = pd.cut(reduced_PerStudentSpending["Per Student Budget"], bins, labels=group_labels)
    
#reduced_PerStudentSpending

In [2034]:
#Drop un-needed columns before you group by
reduced_PerStudentSpending.drop(['Per Student Budget'], axis = 1, inplace = True)

# Fifth Step - Create a GroupBy object based upon "Spending Ranges (Per Student)" column
PerStudentSpending_group = reduced_PerStudentSpending.groupby("Spending Ranges (Per Student)")

# Sixth - Find how many rows fall into each bin - Ensure it counts to 15 school which it does 
#print(PerStudentSpending_group["school"].count())

SchoolSpending_Header = "Scores by School Spending"
print (color.BOLD + SchoolSpending_Header + color.END)
print('*************************')

# Finally - Get the average of each column within the GroupBy object - Get Average score on each 
PerStudentSpending_group.mean()

Scores by School Spending
*************************


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
< $585,83.455399,83.933814,100.000000,100.000000,100.00000
$585-615,83.599686,83.885211,100.000000,100.000000,100.00000
$615-645,79.079225,81.891436,92.636050,96.264069,94.45006
$645-675,76.997210,81.027843,89.041475,94.520946,91.78121


In [2035]:
# For Scores By School Size 
#First - Extract the columns needed for binning from both the school dataframe
School_Size_Type_Performance.head()

reduced_PerSchoolSize = School_Size_Type_Performance.loc[:, ['school','Total Students']]
#reduced_PerSchoolSize.head()

In [2036]:
#Extract the columns needed from the aggregated student dataframe 

reduced_PerSchoolScores = PerStudentSpending_BySchool.loc[:, ['school','Average Math Score', 'Average Reading Score',
                                                               '% Passing Math', '% Passing Reading', 
                                                               '% Overall Passing Rate']]
#reduced_PerSchoolScores.head()

In [2037]:
#Merge to get the combined dataframe that has School Size with performance scores 
merged_SchoolSize_SchoolScores = pd.merge(reduced_PerSchoolSize , reduced_PerSchoolScores, on="school")

#merged_SchoolSize_SchoolScores.head()

In [2038]:
# Now create the bins to slice the data in the dataframe 

#Parse it to find max Total Students and min Total Students 
#print(merged_SchoolSize_SchoolScores["Total Students"].max())

#print(merged_SchoolSize_SchoolScores["Total Students"].min())

In [2039]:
#Put then in Bins and find average for each data within those bins. TO do this: 

#First - Create bins in which to place values based upon Total Students
bins = [0,1000,2000,5000]

# Second - Create labels for these bins
group_labels = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]

In [2040]:
# Third - Slice the Total Students Data and place it into bins
#pd.cut(merged_SchoolSize_SchoolScores["Total Students"], bins, labels=group_labels).head()

In [2041]:
# Fourth - Place the data series into a new column inside of the DataFrame
merged_SchoolSize_SchoolScores["School Size"] = pd.cut(merged_SchoolSize_SchoolScores["Total Students"], bins, labels=group_labels)
    
#merged_SchoolSize_SchoolScores

In [2042]:
# Fifth Step - Drop the Total Students column from the list 
merged_SchoolSize_SchoolScores.drop(['Total Students'], axis = 1, inplace = True)

# Sixth Step - Create a GroupBy object based upon "Total Students" column
SchoolSizeScores_group = merged_SchoolSize_SchoolScores.groupby("School Size")

# Seventh Step - Find how many rows fall into each bin - Ensure it counts to 15 school which it does 
#print(SchoolSizeScores_group["school"].count())

SchoolSize_Header = "Scores by School Size"
print (color.BOLD + SchoolSize_Header + color.END)
print('*************************')

# Finally - Get the average of each column within the GroupBy object - Get Average score on each 
SchoolSizeScores_group.mean()

Scores by School Size
*************************


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,100.000000,100.000000,100.000000
Medium (1000-2000),83.374684,83.864438,100.000000,100.000000,100.000000
Large (2000-5000),77.746417,81.344493,90.367591,95.143406,92.755499


In [2043]:
# For Scores By School Type 
#First - Extract the columns needed for binning from both the school dataframe
#School_Size_Type_Performance.head()

reduced_PerSchoolType = School_Size_Type_Performance.loc[:, ['school','School Type']]
#reduced_PerSchoolType.head()

In [2044]:
#Merge to get the combined dataframe that has School Type with performance scores 
merged_SchoolType_SchoolScores = pd.merge(reduced_PerSchoolType , reduced_PerSchoolScores, on="school")

#merged_SchoolType_SchoolScores

In [2045]:
# First Step - Drop the schools column from the list - Now you have the data needed to get School Type with Scores
merged_SchoolType_SchoolScores.drop(['school'], axis = 1, inplace = True)


In [2046]:
# Second - Group By School Type and find mean values for each 

AverageScore_BySchoolType = pd.DataFrame(merged_SchoolType_SchoolScores.groupby("School Type").mean())

SchoolType_Header = "Scores by School Type"
print (color.BOLD + SchoolType_Header + color.END)
print('**********************')
AverageScore_BySchoolType

Scores by School Type
**********************


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,100.000000,100.000000,100.00000
District,76.956733,80.966636,88.991533,94.449607,91.72057
